In [551]:
!pip install requests
!pip install autopep8
!pip install  urllib
!pip install  streamlit

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [552]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import pandas as pd
import altair as alt
import streamlit as st
import base64

In [553]:
#Auth dashbord https://developer.spotify.com/dashboard/applications
#Env value setting => open jupyter  https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html
# client_id = os.environ.get("CLIENTID")
# client_secret=os.environ.get("CLIENTSEC")
from dotenv import load_dotenv
load_dotenv()
if 'client_id' and 'client_secret'in os.environ:
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
else:
    client_id = os.environ('client_id')
    client_secret = os.environ('client_secret')


In [554]:
client_creds=f"{client_id}:{client_secret}"
type(client_creds)

str

In [555]:
#VALUE: Basic <base64 encoded client_id:client_secret>"
import base64
client_creds=f"{client_id}:{client_secret}"
print(type(client_creds))
     
client_creds_b64=base64.b64encode(client_creds.encode()) 
client_creds_b64

<class 'str'>


b'N2VjMGI3NzUwMWNmNDEwOWE1ZjI3ODVhMzAwNjM0MWQ6YWY0YWU5N2M4ZWFjNDc2ODgxNzdhMmY0YjNhNTlhNDE='

In [556]:
#Request Access Token
#Flow https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
token_url='https://accounts.spotify.com/api/token'
method="POST"

token_data={
    "grant_type": 'client_credentials'
}

# request must include the following HTTP headers:,Value
#header_parameter

token_headers={
    "Authorization": f"Basic {client_creds_b64.decode()}"#Basic <base64 encoded client_id:client_secret>"
}


In [557]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)


{'access_token': 'BQBWDBDY8s7zu5CiMIrDkIr7ohgERMafBiB_f7w8AJ7-aH9tpb4l6LOV1erd1f_9ELsX-kS1akuFzWflLCA', 'token_type': 'Bearer', 'expires_in': 3600}


In [558]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)

if valid_request:
    token_response_data=r.json()
    now=datetime.datetime.now()
    access_token=token_response_data["access_token"]
    exprires_in=token_response_data['expires_in']
    expires=now+datetime.timedelta(seconds=exprires_in)
    didi_expires=expires < now



{'access_token': 'BQC5Eyi1ntby9-Xku0nElPeFLrVo1DxySA5axJLB4EFAQu19ZjA81s17G34igCoktQoreAFQhItnG-rsDxk', 'token_type': 'Bearer', 'expires_in': 3600}


In [559]:
#class
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
                return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [560]:
spotify=SpotifyAPI(client_id,client_secret)

In [561]:
spotify.perform_auth()

True

In [562]:
access_token=spotify.access_token

In [563]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q": "kill bill", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=kill+bill&type=track
https://api.spotify.com/v1/search?q=kill+bill&type=track
200


In [564]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=kill+bill&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3KDKJ7Z0kgJrQS1CH43PM4'},
       'href': 'https://api.spotify.com/v1/artists/3KDKJ7Z0kgJrQS1CH43PM4',
       'id': '3KDKJ7Z0kgJrQS1CH43PM4',
       'name': 'Hatesonny',
       'type': 'artist',
       'uri': 'spotify:artist:3KDKJ7Z0kgJrQS1CH43PM4'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      '

In [565]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import base64
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
if 'client_id' and 'client_secret'in os.environ:
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
else:
    client_id = os.environ('client_id')
    client_secret = os.environ('client_secret')

#class+search method
#another search method type https://developer.spotify.com/documentation/web-api/reference/#/operations/search


class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
            raise Exception("Not authenticate client")
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
                
        token=self.access_token
        expires=self.access_token_expires
        now=datetime.datetime.now()
        
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token==None:
            self.perform_auth()
            return self.get_access_token
            
        return token
    
    #header
    def get_resource_header(self):
            access_token = self.get_access_token()
            headers = {
                "Authorization": f"Bearer {access_token}"
            }
            return headers

    #depending on input data, get Json
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    #album json
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    #artist json 
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    #research Json
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    #query : dic or string  %20 is space
    def search(self,query=None, operator=None,operator_query=None ,search_type='track'):
        if query == None:
               raise Exception("A query is required")
               
        if isinstance(query,dict):
               query=" ".join([f"{key},{value}"for key,value in query.items()])#inline 
                
        if operator != None and operator_query!=None:
            operator=operator.upper()
            if operator== "or" or operator.lower=="not":
                operator=operator.upper()
                if isinstance(operator_query,str):
                    query=f"{query}{operator}{operator_query}"
               
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    
    
        #playlist_json_data
    def playlist_search_json_createdata(self,query="tatanic"):
        playlists_json=self.search(query=query,search_type="playlist")

        #Data
        need_playlist= []
        for i, item in enumerate(playlists_json["playlists"]["items"]):
            need_playlist.append((i,
                          item["name"], 
                          item["external_urls"]["spotify"],
                          item["id"],
                          item["images"][0]["url"],
                         
         ))
            playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
        return playlist_df
    
    
    def spotify_get_organised_data(self,Name_of_Movie):
            #Data
        Data = self.search({"album": f"{Name_of_Movie}"}, search_type="track")
        
        #need contents 
        need = []
        for i, item in enumerate(Data['tracks']['items']):
            track = item['album']
            track_id = item['id']
            song_name = item['name']
            popularity = item['popularity']
            need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))

        #data frame top10
        Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity',"albumID"))

        access_token = self.access_token
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/audio-features/"

        #Full data
        Feat_df = pd.DataFrame()
        for id in Track_df['Id'].iteritems():
            track_id = id[1]
            lookup_url = f"{endpoint}{track_id}"
            ra = requests.get(lookup_url, headers=headers)
            audio_feat = ra.json()
            Features_df = pd.DataFrame(audio_feat, index=[0])
            Feat_df = Feat_df.append(Features_df)
        Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")

        #sort
        Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)

        #chart df 
        chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
        Name_of_Feat="energy"

        return chart_df
    
 #playlist_json_data
    def playlist_search_json_createdata(self,query=None):
        query="titanic" if query is None else query
        
        playlists_json=self.search(query=query,search_type="playlist")
        
        #Data
        need_playlist= []
        for i, item in enumerate(playlists_json["playlists"]["items"]):
            need_playlist.append((i,
                          item["name"], 
                          item["external_urls"]["spotify"],
                          item["id"],
                          item["images"][0]["url"],
                         
         ))
            playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
        return playlist_df
        

In [566]:
spotify=SpotifyAPI(client_id,client_secret)

In [567]:
spotify.spotify_get_organised_data("titanic")["albumID"]

q=album%2Ctitanic&type=track


/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/3483318123.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/3483318123.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/3483318123.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)


1    3XwgFTxiWxep4s1VjPEaHs
0    24YoVjBTadmRW2Zhc9ou9S
2    2mgkd00677MkIIw09g0kss
Name: albumID, dtype: object

In [568]:
#playlist json 
name="spiderman"
playlists_json=spotify.search(query=name,search_type="playlist")
playlists_json


q=spiderman&type=playlist


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=spiderman&type=playlist&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': '',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/4iFrBplODblDV2Gg0zvbBl'},
    'href': 'https://api.spotify.com/v1/playlists/4iFrBplODblDV2Gg0zvbBl',
    'id': '4iFrBplODblDV2Gg0zvbBl',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706c0000bebb4a23cd64ff93df00bae5702f',
      'width': None}],
    'name': 'Spiderman - Alle Hörspiele',
    'owner': {'display_name': 'Disney Hörspiele',
     'external_urls': {'spotify': 'https://open.spotify.com/user/1rt8f3lznvh0p1ct3798qly5a'},
     'href': 'https://api.spotify.com/v1/users/1rt8f3lznvh0p1ct3798qly5a',
     'id': '1rt8f3lznvh0p1ct3798qly5a',
     'type': 'user',
     'uri': 'spotify:user:1rt8f3lznvh0p1ct3798qly5a'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'Nyw5MWQ2YzhkMWEwYTA5NWM3ZWNiOWE2YjI3ZGNlN2Y0ZTV

In [569]:
playlist_df=spotify.playlist_search_json_createdata(query=name)
playlist_df

q=spiderman&type=playlist


,item,Name,PlaylistURL,ID,ImageURL
0,0,Spiderman - Alle Hörspiele,https://open.spotify.com/playlist/4iFrBplODblD...,4iFrBplODblDV2Gg0zvbBl,https://i.scdn.co/image/ab67706c0000bebb4a23cd...
1,1,Spider-man🕷,https://open.spotify.com/playlist/1NIahjNZYEDE...,1NIahjNZYEDE9QQQNC7Vy5,https://i.scdn.co/image/ab67706c0000bebbe4eeb2...
2,2,Spider-Man: No Way Home Soundtrack,https://open.spotify.com/playlist/6nwPqanMXhuG...,6nwPqanMXhuGACQZliH4gP,https://i.scdn.co/image/ab67706c0000bebb26bac1...
3,3,spider-man vibes,https://open.spotify.com/playlist/2INWocVLeGaF...,2INWocVLeGaF5HvHoB2KFD,https://i.scdn.co/image/ab67706c0000bebbe80493...
4,4,Spiderman: Into the Spider Verse Soundtrack,https://open.spotify.com/playlist/7JDCGxFthByJ...,7JDCGxFthByJYALY0Ui5z0,https://i.scdn.co/image/ab67616d0000b273e2e352...
5,5,Miles Morales Soundtrack (Spiderman),https://open.spotify.com/playlist/7KtG0weK8vwS...,7KtG0weK8vwSjjSd5Cvokg,https://i.scdn.co/image/ab67706c0000bebb5ef5e8...
6,6,Spiderman,https://open.spotify.com/playlist/608BC68sI9NZ...,608BC68sI9NZeANHJsaWRf,https://mosaic.scdn.co/640/ab67616d0000b2733c8...
7,7,SpiderMan: No Way Home (All Songs),https://open.spotify.com/playlist/4kR1mRs8ih8n...,4kR1mRs8ih8nOU4zKwfF4Z,https://i.scdn.co/image/ab67706c0000bebb1f4615...
8,8,spider-man swinging music,https://open.spotify.com/playlist/2ub0l9XVMuNo...,2ub0l9XVMuNoAm7oN3CRwe,https://i.scdn.co/image/ab67706c0000bebbf1d6a9...
9,9,Spiderman swinging music,https://open.spotify.com/playlist/2TAVgn06KBpC...,2TAVgn06KBpCmRSinG9Ug7,https://i.scdn.co/image/ab67706c0000bebb6e93d5...


In [570]:
#top URL
playlist_URI=playlist_df["PlaylistURL"][0]
playlist_URI

'https://open.spotify.com/playlist/4iFrBplODblDV2Gg0zvbBl'

In [571]:
#spotipy recommendation
!pip install  spotipy

In [572]:
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
sp

In [573]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]

In [574]:
sp.audio_features(track_uri)[0]

{'danceability': 0.641,
 'energy': 0.712,
 'key': 7,
 'loudness': -7.701,
 'mode': 1,
 'speechiness': 0.917,
 'acousticness': 0.465,
 'instrumentalness': 0,
 'liveness': 0.48,
 'valence': 0.759,
 'tempo': 134.827,
 'type': 'audio_features',
 'id': '2pCRsgW9HIC0Px4hWgltSV',
 'uri': 'spotify:track:2pCRsgW9HIC0Px4hWgltSV',
 'track_href': 'https://api.spotify.com/v1/tracks/2pCRsgW9HIC0Px4hWgltSV',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2pCRsgW9HIC0Px4hWgltSV',
 'duration_ms': 84787,
 'time_signature': 4}

In [577]:
#roubust
# spotify.search(query="titanic",operator="NOT",operator_query="Zone",search_type="track")

In [584]:
movie_name="titanic"
data = spotify.search({"album": f"{movie_name}"}, search_type="track")
data

q=album%2Ctitanic&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=album%2Ctitanic&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PhL2Vdao2v8SS8AptuhAr'},
       'href': 'https://api.spotify.com/v1/artists/3PhL2Vdao2v8SS8AptuhAr',
       'id': '3PhL2Vdao2v8SS8AptuhAr',
       'name': 'James Horner',
       'type': 'artist',
       'uri': 'spotify:artist:3PhL2Vdao2v8SS8AptuhAr'}],
     'available_markets': ['US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/24YoVjBTadmRW2Zhc9ou9S'},
     'href': 'https://api.spotify.com/v1/albums/24YoVjBTadmRW2Zhc9ou9S',
     'id': '24YoVjBTadmRW2Zhc9ou9S',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273065c3565b58f738e30035f10',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02065c3565b58f738e30035f10',
       'width': 300},
      {'height

In [586]:
#trackdata
need = []
for i, item in enumerate(data['tracks']['items']):
    track = item['album']
    track_id = item['id']
    song_name = item['name']
    popularity = item['popularity']
    need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))
    
need 

[(0,
  'James Horner',
  "Titanic: Original Motion Picture Soundtrack - Collector's Anniversary Edition",
  '7ICIp5yx2L8x5ymAZugi3D',
  'A Building Panic - Album Suite',
  '1997',
  20,
  '24YoVjBTadmRW2Zhc9ou9S'),
 (1,
  'James Horner',
  "Titanic: Original Motion Picture Soundtrack - Collector's Anniversary Edition",
  '6qFFrrMou6AEBZse0k9fFK',
  'A Building Panic - Album Suite',
  '1997',
  30,
  '3XwgFTxiWxep4s1VjPEaHs'),
 (2,
  'James Horner',
  'The Rocketeer (Original Motion Picture Soundtrack)',
  '1fNu1TRJtLJsZ6ksALUh5l',
  'Rocketeer to the Rescue/End Title - 1991 Soundtrack Album/2020 Remaster',
  '2021-05-28',
  17,
  '2mgkd00677MkIIw09g0kss')]

In [587]:
Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity',"albumID"))
Track_df .head(10)

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,albumID
0,0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,7ICIp5yx2L8x5ymAZugi3D,A Building Panic - Album Suite,1997,20,24YoVjBTadmRW2Zhc9ou9S
1,1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,6qFFrrMou6AEBZse0k9fFK,A Building Panic - Album Suite,1997,30,3XwgFTxiWxep4s1VjPEaHs
2,2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,1fNu1TRJtLJsZ6ksALUh5l,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17,2mgkd00677MkIIw09g0kss


In [588]:
access_token = spotify.access_token

headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/audio-features/"

In [589]:
Feat_df = pd.DataFrame()
for id in Track_df['Id'].iteritems():
    track_id = id[1]
    lookup_url = f"{endpoint}{track_id}"
    ra = requests.get(lookup_url, headers=headers)
    audio_feat = ra.json()
    Features_df = pd.DataFrame(audio_feat, index=[0])
    Feat_df = Feat_df.append(Features_df)
Feat_df

/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.0865,0.233,2,-19.175,1,0.0461,0.600,0.775,0.0916,0.0360,63.299,audio_features,7ICIp5yx2L8x5ymAZugi3D,spotify:track:7ICIp5yx2L8x5ymAZugi3D,https://api.spotify.com/v1/tracks/7ICIp5yx2L8x...,https://api.spotify.com/v1/audio-analysis/7ICI...,488933,4
0,0.0859,0.233,10,-19.174,1,0.0462,0.600,0.768,0.0916,0.0361,66.117,audio_features,6qFFrrMou6AEBZse0k9fFK,spotify:track:6qFFrrMou6AEBZse0k9fFK,https://api.spotify.com/v1/tracks/6qFFrrMou6AE...,https://api.spotify.com/v1/audio-analysis/6qFF...,488933,3
0,0.1270,0.208,0,-17.739,1,0.0378,0.675,0.921,0.0851,0.0692,94.606,audio_features,1fNu1TRJtLJsZ6ksALUh5l,spotify:track:1fNu1TRJtLJsZ6ksALUh5l,https://api.spotify.com/v1/tracks/1fNu1TRJtLJs...,https://api.spotify.com/v1/audio-analysis/1fNu...,394274,4


In [590]:
Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")
Full_Data 

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,albumID,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,7ICIp5yx2L8x5ymAZugi3D,A Building Panic - Album Suite,1997,20,24YoVjBTadmRW2Zhc9ou9S,0.0865,0.233,...,0.0916,0.0360,63.299,audio_features,7ICIp5yx2L8x5ymAZugi3D,spotify:track:7ICIp5yx2L8x5ymAZugi3D,https://api.spotify.com/v1/tracks/7ICIp5yx2L8x...,https://api.spotify.com/v1/audio-analysis/7ICI...,488933,4
1,1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,6qFFrrMou6AEBZse0k9fFK,A Building Panic - Album Suite,1997,30,3XwgFTxiWxep4s1VjPEaHs,0.0859,0.233,...,0.0916,0.0361,66.117,audio_features,6qFFrrMou6AEBZse0k9fFK,spotify:track:6qFFrrMou6AEBZse0k9fFK,https://api.spotify.com/v1/tracks/6qFFrrMou6AE...,https://api.spotify.com/v1/audio-analysis/6qFF...,488933,3
2,2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,1fNu1TRJtLJsZ6ksALUh5l,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17,2mgkd00677MkIIw09g0kss,0.1270,0.208,...,0.0851,0.0692,94.606,audio_features,1fNu1TRJtLJsZ6ksALUh5l,spotify:track:1fNu1TRJtLJsZ6ksALUh5l,https://api.spotify.com/v1/tracks/1fNu1TRJtLJs...,https://api.spotify.com/v1/audio-analysis/1fNu...,394274,4


In [591]:
Full_Data ["track_href"][0]

'https://api.spotify.com/v1/tracks/7ICIp5yx2L8x5ymAZugi3D'

In [ ]:
Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)
Sort_DF

In [ ]:
chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
chart_df

In [ ]:
chart_df.drop_duplicates(subset=['Album Name'], keep="first")

In [ ]:
Name_of_Feat="energy"
st.header(f"Popularity+Energy Chart")
c = alt.Chart(chart_df).mark_circle().encode(
    alt.X('Popularity', scale=alt.Scale(zero=False)), y=f'{Name_of_Feat}', color=alt.Color('Popularity', scale=alt.Scale(type='log',scheme='rainbow')), 
    size=alt.value(300), tooltip=['Popularity', f'{Name_of_Feat}', 'Song Name', 'Album Name']).interactive()

c

In [ ]:
st.header(f"{movie_name}sound track list")
st.table(chart_df)

In [ ]:
#Music play feature 
drop_deplicated_data=chart_df.drop_duplicates(subset=['Album Name'], keep="first")

In [ ]:
chart_df["albumID"].reset_index(drop=True)

In [ ]:
chart_df["albumID"].reset_index(drop=True)[0]

In [ ]:
for i in range(len(chart_df["albumID"])):
    uri=chart_df["albumID"].reset_index(drop=True)[i]
    print(f"https://open.spotify.com/embed/album/{uri}")

In [ ]:
drop_deplicated_data=chart_df.drop_duplicates(subset=['Album Name'], keep="first").reset_index(drop=True)
drop_deplicated_data

In [ ]:
#spotify_get_organised_data class function test
spotify_get_data=spotify.spotify_get_organised_data("titanic")
spotify_get_data

In [ ]:
!pip install streamlit

In [ ]:
#get top1
spotify_get_data["albumID"][0]

In [ ]:
import streamlit.components.v1 as components

uri=spotify_get_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )    

In [ ]:
spotify_get_data=SpotifyAPI.spotify_get_organised_data(movie_name)
spotify_get_data["albumID"][0]

uri=spotify_get_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )    

In [ ]:
uri=spotify.spotify_get_organised_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )  

In [ ]:
import streamlit.components.v1 as components

uri=spotify.spotify_get_organised_data["albumID"][0]
# components.html(
#             f"""
#             <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
#             allow="encrypted-media" ></iframe
#                 """,
#                 height=400
#             )  